<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso1/ciclo4/2_word_tagging_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1Q6vQcIWFPY27isBepABpJ7nroUNKox_Z" width="100%">

# **Etiquetado de Palabras**
---

En este notebook veremos cómo podemos clasificar tokens de forma automática con `spacy`.

Comenzamos importando las librerías necesarias:

In [ ]:
import spacy
import pandas as pd
from tqdm import tqdm # Sirve para visualizar una barra de progreso
from IPython.display import display

## **1. Descripción General**
---

`spacy` nos permite entrenar modelos personalizados y crear nuestros propios _Pipeline_ para tareas específicas. Esto es especialmente útil cuando deseamos realizar tareas específicas en nuestro dominio.

El proceso de entrenamiento de `spacy` se puede describir de acuerdo a la siguiente figura:

<img src="https://drive.google.com/uc?export=view&id=1a-lk4nZMe7Vqx4cR_v6Cejhneibc3QOr" width="80%">

En este caso, se usan datos de entrenamiento compuestos de textos y etiquetas (normalmente para clasificación de secuencias) y por medio de técnicas de optimización basadas en gradiente se entrena un modelo basado en redes neuronales. Dicho modelo posteriormente se usa para hacer predicciones automáticas y se puede almacenar y exportar.

En este notebook veremos un ejemplo de reconocimiento de entidades nombradas (NER) personalizado, donde buscaremos etiquetar de forma automática textos en categorías específicas. Por ejemplo:

<img src="https://drive.google.com/uc?export=view&id=10hr8FWCVUKb-fN3nSiWa0l3jjO-1E0hi" width="80%">

## **2. Conjunto de Datos de NER**
---

Es importante saber estructurar el conjunto de datos para poder entrenar un modelo personalizado de `spacy`, en este caso usaremos el conjunto de datos [Entity Annotated Corpus](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus) de Kaggle.

Comenzamos cargándolo:

In [ ]:
data = pd.read_parquet("https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u4/ner_dataset.parquet")
display(data.head())

Como podemos ver, tenemos 4 columnas:

- `sentence`: identificador de la oración.
- `token`: palabra dentro del texto.
- `pos`: etiqueta de tipo _POS_.
- `ner`: etiqueta de tipo _NER_.

Veamos qué etiquetas de tipo _NER_ tiene el conjunto de datos:

In [ ]:
labels = data.ner.unique()
display(labels)

Estas corresponden a:

- `o`: ninguna entidad asociada.
- `geo`: lugar geográfico.
- `gpe`: entidad geopolítica.
- `per`: persona.
- `org`: organización.
- `tim`: indicador de tiempo.
- `art`: artefacto.
- `nat`: fenómeno natural.
- `eve`: evento.

Podemos extraer una oración del texto de la siguiente forma:

In [ ]:
sent_id = 1
text = " ".join(
        data
        .query(f"sentence == '{sent_id}'")
        .token.to_list()
        )
display(text)

También podemos ver las entidades nombradas asociadas a este texto:

In [ ]:
sent_id = 1
text = " ".join(
        data
        .query(f"sentence == '{sent_id}'")
        .ner.to_list()
        )
display(text)

Para entrenar un modelo de `spacy` debemos extraer el texto completo y asociar cada etiqueta a su posición.

Definimos la función `convert_sentence` para transformar cada oración del corpus en un formato compatible con `spacy`:

In [ ]:
def convert_sentence(df):
    sentence = " ".join(df["token"]) # extraemos el texto completo
    tags = []
    pos = 0
    for _, row in df.iterrows():
        if row["ner"] != 'o': # Filtramos los tokens con entidad
            tags.append(
                    (pos, pos + len(row["token"]), row["ner"])
                    )
        pos += len(row["token"]) + 1 # asignamos la posición de cada token.
    return (sentence, {"entities": tags})

Veamos un ejemplo:

In [ ]:
sent = data.query("sentence == '1'")
sent_conv = convert_sentence(sent)
display(sent_conv)

Podemos ver que una oración se codifica como una tupla de dos valores:

1. El texto completo.
2. Un diccionario donde se establece el inicio, el final y el tipo de una entidad dentro del texto.

Podemos aplicar esta función a cada una de las oraciones del texto de la siguiente forma:

In [ ]:
corpus = (
        data
        .groupby("sentence")
        .apply(convert_sentence)
        .tolist()
        )
display(corpus[:2])

> **Nota**: como puede ver, estamos usando un conjunto de datos con etiquetas personalizadas. Si desea realizar cualquier tarea de clasificación de tokens lo puede hacer siempre que cuente con un conjunto de datos etiquetado de una forma similar al que tenemos.

## **3. Pipeline Personalizado de Spacy**
---

Podemos definir un _Pipeline_ personalizado para cualquier lenguaje soportado por `spacy`. Recuerde que hay reglas (como tokenizado, stopwords, entre otras) que son propias del lenguaje y no requieren ningún modelo específico.

En este caso definiremos un _Pipeline_ en blanco para inglés:

In [ ]:
nlp = spacy.blank("en")
display(nlp)

Veamos los componentes que tiene este _Pipeline_:

In [ ]:
display(nlp.component_names)

Como podemos ver, hasta el momento no debería tener ningún componente. Vamos a agregar un componente de `ner`:

In [ ]:
ner = nlp.add_pipe("ner")

Ahora, podemos validar que exista este componente:

In [ ]:
display(nlp.component_names)

El componente `ner` debe saber qué etiquetas debería clasificar. Estas las podemos obtener del `DataFrame`:

In [ ]:
labels = data.ner.unique()
labels = labels[labels != 'o']
display(labels)

Agregamos estas etiquetas al `ner`:

In [ ]:
for label in labels:
    ner.add_label(label)

Podemos validarlo:

In [ ]:
display(ner.labels)

## **4. Entrenamiento**
---

Para el entrenamiento del _Pipeline_ personalizado, debemos usar los siguientes tres elementos de `spacy`:

<img src="https://drive.google.com/uc?export=view&id=1gXNxwthy-lJtpbcQhabFs_R6Bof10YjN" width="80%">

- `minibatch`: el modelo es entrenado por lotes, es decir, no se utiliza el corpus completo para entrenar el modelo, sino que se usa un grupo pequeño de documentos (batch) para ir ajustándolo de forma iterativa. Esto permite que el modelo pueda ser entrenado con grandes cantidades de datos. El `minibatch` nos permite definir este subconjunto de datos.
- `compounding`: se trata de una estrategia para modificar de forma iterativa el tamaño del batch en el entrenamiento. En este caso, requiere un número mínimo y máximo de oraciones en lote y una tasa para modificar el tamaño del batch de forma incremental.
- `Example`: define una muestra en un formato compatible con `spacy`.

Importamos estos elementos:

In [ ]:
from spacy.util import minibatch, compounding
from spacy.training import Example

Ahora, creamos los batches:

In [ ]:
batches = minibatch(
        items = corpus,
        size = compounding(
            start=4, stop=32, compound=1.01
            )
        )
display(batches)

El resultado es un `generator` de _Python_, este objeto nos permite extraer batches de forma iterativa. Veamos un ejemplo:

In [ ]:
batch = next(batches)
display(len(batch))
display(batch)

Como podemos ver, obtenemos 4 oraciones, lo cual corresponde al tamaño inicial del `compounding` que definimos.

Ahora, podemos definir el entrenamiento del modelo completo, para ello vamos a definir los siguientes hiperparámetros:

- `ITERS`: número de iteraciones de entrenamiento del modelo.
- `LR`: tasa de aprendizaje, permite controlar el sobre y subajuste del modelo (de esto se habla con mayor profundidad en el módulo de _Deep Learning_).
- `DROP`: regularización de redes neuronales, toma valores entre 0 y 1 (de esto se habla con mayor profundidad en el módulo de _Deep Learning_).

In [ ]:
ITERS = 100
LR = 1e-3
DROP = 0.3

Ahora, definimos un optimizador (el que se encarga de entrenar el modelo):

In [ ]:
optimizer = nlp.begin_training()
display(optimizer)

Especificamos la tasa de aprendizaje como atributo del optimizador:

In [ ]:
optimizer.learn_rate = LR

Finalmente, definimos una función que se encargará del entrenamiento:

In [ ]:
def train(nlp, batches, n_iter, optimizer, drop):
    # Estructura donde se almacenarán las pérdidas del modelo.
    losses = {}
    # Pérdida acumulada
    loss = 0
    # Barra de progreso
    pbar = tqdm(range(n_iter))
    # Iteramos por un número de iteraciones
    for _ in pbar:
        # Extraemos un batch de datos:
        batch = next(batches)

        # Convertimos el corpus a ejemplos de spacy.
        examples = []
        for case in batch:
            example = Example.from_dict(
                    nlp.make_doc(case[0]), case[1]
                    )
            examples.append(example)
        # Ajustamos el modelo con el batch de datos
        nlp.update(
            examples,
            sgd=optimizer,
            drop=drop,
            losses=losses
            )

        # Imprimimos la pérdida del modelo.
        cur_loss = losses["ner"] - loss
        pbar.set_description(f"Loss: {cur_loss}")
        loss = losses["ner"]

Ahora, entrenamos el modelo:

In [ ]:
train(nlp, batches, ITERS, optimizer, DROP)

## **5. Aplicación**
---

Por último, veamos el modelo en funcionamiento, podemos extraer una oración del corpus:

In [ ]:
text = corpus[0][0]
display(text)

Podemos crear un documento de `spacy`:

In [ ]:
doc = nlp(text)

Finalmente, veamos las entidades:

In [ ]:
doc.ents

También podemos ver sus tipos:

In [ ]:
for token in doc:
    if token.ent_type_ :
        display(f"{token.text} - {token.ent_type_}")

También es posible usar `displacy` para visualizar las entidades:

In [ ]:
from spacy import displacy

Veamos un gráfico de entidades:

In [ ]:
svg = displacy.render(doc, jupyter=True, style="ent")

Por último, podemos exportar el _Pipeline_ personalizado con el método `to_disk`:

In [ ]:
nlp.to_disk("mypipe")

Esto genera una carpeta con todos los componentes del modelo. Podemos visualizarlos con la utilidad `tree`:

In [ ]:
!apt install tree

In [ ]:
!tree mypipe/

Este _Pipeline_ se puede cargar como cualquier otro modelo de `spacy`:

In [ ]:
nlp = spacy.load("mypipe")
display(nlp)

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [Training Pipelines & models](https://spacy.io/usage/training).
- [Thinc for deep learning](https://thinc.ai/).
- _Fuente de los íconos_
     - Flaticon. Justify free icon [PNG]. https://www.flaticon.com/free-icon/justify_6935287


## Créditos
---

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*